## Case 1 code + logic

In [ ]:
import pandas as pd

class IndirectRouteOptimizer:
    def __init__(self, shipment_data, airport_volume, flight_time_matrix,
                 branch_hub_time, hub_to_airport_time, hub_to_hub_time):
        self.shipment_data = shipment_data
        self.airport_volume = airport_volume
        self.flight_time_matrix = flight_time_matrix
        self.branch_hub_time = branch_hub_time
        self.hub_to_airport_time = hub_to_airport_time
        self.hub_to_hub_time = hub_to_hub_time

        self._build_lookups()

    def _build_lookups(self):
        self.branch_hub_lookup = {(r['Branch'], r['Hub']): r['Duration_hr']
                                  for _, r in self.branch_hub_time.dropna().iterrows()}
        self.hub_airport_lookup = {(r['Hub'], r['Final_Airport_City']): r['Duration_hr']
                                   for _, r in self.hub_to_airport_time.dropna().iterrows()}
        self.hub_hub_lookup = {(r['Hub1'], r['Hub2']): r['Duration_hr']
                               for _, r in self.hub_to_hub_time.dropna().iterrows()}
        self.flight_time_lookup = {(r['Origin'], r['Destination']): r['Estimated_Flight_Time_Minutes'] / 60
                                   for _, r in self.flight_time_matrix.dropna().iterrows()}
        self.airport_volume_lookup = {(r['Origin_Airport'], r['Destination_Airport']): r['Volume_kg']
                                      for _, r in self.airport_volume.dropna().iterrows()}

        # Airport to nearest hub (for middle airport)
        self.airport_to_nearest_hub = {}
        for airport in self.hub_to_airport_time['Final_Airport_City'].unique():
            subset = self.hub_to_airport_time[self.hub_to_airport_time['Final_Airport_City'] == airport]
            subset = subset.dropna(subset=['Duration_hr'])
            if not subset.empty:
                best = subset.loc[subset['Duration_hr'].idxmin()]
                self.airport_to_nearest_hub[airport] = (best['Hub'], best['Duration_hr'])

    def get_time(self, d, key):
        return d.get(key, None)

    def compute_indirect_path(self, row):
        origin_branch = row['Origin Branch']
        origin_hub = row['Origin Hub']
        origin_airport = row['Origin Airport']
        dest_branch = row['Destination Branch']
        dest_hub = row['Destination Hub']
        dest_airport = row['Destination Airport']
        product_type = row['New Product GCP']
        weight = row['Total Weight']

        # Fixed segment times, handle hub = branch edge case
        t1 = 0 if origin_branch == origin_hub else self.get_time(self.branch_hub_lookup, (origin_branch, origin_hub))
        t3 = self.get_time(self.hub_airport_lookup, (origin_hub, origin_airport))
        t13 = 0 if dest_branch == dest_hub else self.get_time(self.branch_hub_lookup, (dest_branch, dest_hub))

        if None in [t1, t3, t13]:
            return pd.Series([None] * 7)

        best_route, best_time, best_volume, best_path = None, float('inf'), 0, ""

        for middle_airport in set([dest for (_, dest), vol in self.airport_volume_lookup.items() if vol >= 100]):
            if middle_airport in [origin_airport, dest_airport]:
                continue

            t5 = self.get_time(self.flight_time_lookup, (origin_airport, middle_airport))
            if t5 is None:
                continue

            middle_hub, t7 = self.airport_to_nearest_hub.get(middle_airport, (None, None))
            if not middle_hub or t7 is None:
                continue

            t9 = self.get_time(self.hub_hub_lookup, (middle_hub, dest_hub))
            if t9 is None:
                continue

        # Fixed processing times
            t2, t4, t6, t8, t10 = 2.5, 2.0, 2.0, 2.5, 2.5
            t11, t12 = t9, t13

        # Ensure no segment is None
            required_times = [t1, t2, t3, t4, t5, t6, t7, t8, t11, t12]
            if any(v is None for v in required_times):
                continue

            total_time = sum(required_times)
            total_volume = self.airport_volume_lookup.get((origin_airport, middle_airport), 0) + weight

            if total_volume >= 45 and total_time < best_time:
                best_time = total_time
                best_volume = total_volume
                best_path = (
                    f"{origin_branch} → {origin_hub} → {origin_airport} ✈️ {middle_airport} → "
                    f"{middle_hub} → {dest_hub} → {dest_branch}"
                )
                best_route = (t1, t2, t3, t4, t5, t6, t7, t8, t11, t12)

        if best_path:
            return pd.Series([
                product_type, weight, best_path, best_time, best_volume,
                f"t1:{best_route[0]:.2f} t2:{best_route[1]} t3:{best_route[2]:.2f} "
                f"t4:{best_route[3]} t5:{best_route[4]:.2f} t6:{best_route[5]} "
                f"t7:{best_route[6]:.2f} t8:{best_route[7]} t11:{best_route[8]:.2f} t12:{best_route[9]:.2f}",
                "Air Route"
            ])
        else:
            return pd.Series([None] * 7)

    def run_optimizer(self):
        print("🔁 Running indirect route optimizer...")
        required = self.shipment_data[self.shipment_data["Path_Required"] == "YES"].copy()
        output = required.apply(self.compute_indirect_path, axis=1)
        output.columns = [
            "Product Type", "Weight (kg)", "Suggested Route", "Indirect Total Time",
            "Total Volume", "Time_Components", "Recommendation"
        ]
        output.insert(0, "Origin Branch", required["Origin Branch"])
        output.insert(1, "Destination Branch", required["Destination Branch"])
        return output.dropna(subset=["Suggested Route"])


# Example usage
def load_and_run_optimizer():
    shipment_data = pd.read_csv("/content/Daily_datafor_(EP+BP)_ES_path_required_(with_new_rows).csv")
    airport_volume =pd.read_csv("/content/Total_volume_from_airport_to_air(corrected).csv")
    flight_time_matrix = pd.read_csv("/content/acutal_flight_time_matrix.csv")
    branch_hub_time = pd.read_csv("/content/branch_hub_time_haversine_correct.csv")
    hub_to_airport_time = pd.read_csv("/content/Hub_to_airport_time_matrix - Sheet1.csv")
    hub_to_hub_time = pd.read_csv("/content/hub_distance_time_matrix_haversine.csv")

    optimizer = IndirectRouteOptimizer(
        shipment_data,
        airport_volume,
        flight_time_matrix,
        branch_hub_time,
        hub_to_airport_time,
        hub_to_hub_time
    )
    final_result = optimizer.run_optimizer()
    final_result.to_csv("Case1_indirect_for_left_of_the_rows.csv", index=False)
    return final_result

# Uncomment to run:
result = load_and_run_optimizer()
print(result.head())

🔁 Running indirect route optimizer...
        Origin Branch     Destination Branch Product Type  Weight (kg)  \
0  A01-AHMEDABAD APEX        A05-VAPI BRANCH        EP_BP          0.0   
1  A01-AHMEDABAD APEX  A07-ANKLESHWAR BRANCH        EP_BP          0.0   
2  A01-AHMEDABAD APEX     A11-KHOKHRA BRANCH        EP_BP          0.0   
3  A01-AHMEDABAD APEX    A17-BARODA SCS APEX        EP_BP          0.0   
4  A01-AHMEDABAD APEX     A18-MEHSANA BRANCH        EP_BP          0.0   

                                     Suggested Route  Indirect Total Time  \
0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...            16.434128   
1  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...            15.594128   
2  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...            16.719513   
3  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...            18.499513   
4  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...            17.729513   

   Total Volume                                    Tim

## HUB To all airports Matrix

In [ ]:
import pandas as pd
import math
import csv

# Airport coordinates for the requested cities
airports = {
    'AGARTALA': {'lat': 23.8870, 'lng': 91.2403, 'iata': 'IXA'},
    'AHMEDABAD': {'lat': 23.0726, 'lng': 72.6177, 'iata': 'AMD'},
    'BANGALORE': {'lat': 13.1986, 'lng': 77.7066, 'iata': 'BLR'},
    'BHUBANESHWAR': {'lat': 20.2444, 'lng': 85.8178, 'iata': 'BBI'},
    'CHANDIGARH': {'lat': 30.6735, 'lng': 76.7884, 'iata': 'IXC'},
    'CHENNAI': {'lat': 12.9941, 'lng': 80.1709, 'iata': 'MAA'},
    'COCHIN': {'lat': 10.1520, 'lng': 76.4019, 'iata': 'COK'},
    'COIMBATORE': {'lat': 11.0290, 'lng': 77.0434, 'iata': 'CJB'},
    'DELHI': {'lat': 28.5562, 'lng': 77.1000, 'iata': 'DEL'},
    'DIBRUGARH': {'lat': 27.4839, 'lng': 95.0169, 'iata': 'DIB'},
    'GUWAHATI': {'lat': 26.1061, 'lng': 91.5856, 'iata': 'GAU'},
    'HYDERABAD': {'lat': 17.2313, 'lng': 78.4298, 'iata': 'HYD'},
    'INDORE': {'lat': 22.7279, 'lng': 75.8081, 'iata': 'IDR'},
    'JAIPUR': {'lat': 26.8242, 'lng': 75.8122, 'iata': 'JAI'},
    'KOLKATA': {'lat': 22.6546, 'lng': 88.4467, 'iata': 'CCU'},
    'LUCKNOW': {'lat': 26.7606, 'lng': 80.8893, 'iata': 'LKO'},
    'MUMBAI': {'lat': 19.0896, 'lng': 72.8656, 'iata': 'BOM'},
    'NAGPUR': {'lat': 21.0922, 'lng': 79.0472, 'iata': 'NAG'},
    'PATNA': {'lat': 25.5913, 'lng': 85.0880, 'iata': 'PAT'},
    'PUNE': {'lat': 18.5822, 'lng': 73.9197, 'iata': 'PNQ'},
    'RANCHI': {'lat': 23.3141, 'lng': 85.3217, 'iata': 'IXR'},
    'SILIGURI': {'lat': 26.7153, 'lng': 88.3286, 'iata': 'IXB'},
    'SURAT': {'lat': 21.1140, 'lng': 72.7417, 'iata': 'STV'},
    'TRICHY': {'lat': 10.7654, 'lng': 78.7097, 'iata': 'TRZ'},
    'VARANASI': {'lat': 25.4522, 'lng': 82.8597, 'iata': 'VNS'},
    'VIJAYAWADA': {'lat': 16.5308, 'lng': 80.7967, 'iata': 'VGA'}
}

# Hub coordinates from the provided data
hubs = {
    'A01-AHMEDABAD APEX': {'lat': 23.106556, 'lng': 72.683222},
    'A04-SURAT APEX': {'lat': 21.175874, 'lng': 72.955277},
    'B10-BANGALORE YELAHANKA APEX': {'lat': 13.1066, 'lng': 77.6187},
    'C42-GUINDY APEX': {'lat': 13.05947836, 'lng': 80.13138739},
    'E01-COIMBATORE APEX': {'lat': 11.017391, 'lng': 76.98321},
    'E04-SALEM APEX': {'lat': 11.6476208, 'lng': 78.1225439},
    'E08-MADURAI APEX': {'lat': 9.9160699, 'lng': 78.0999871},
    'H13-HYDERABAD SHAMSHABAD APEX': {'lat': 17.25955818, 'lng': 78.37652436},
    'I01-INDORE APEX': {'lat': 22.746654, 'lng': 75.892436},
    'J10-AMBALA APEX': {'lat': 30.403561, 'lng': 76.787836},
    'J29-PANCHKULA APEX': {'lat': 30.667815, 'lng': 76.837394},
    'K16-KOLKATA RAJARHAT APEX': {'lat': 22.62824545, 'lng': 88.44588786},
    'K66-SILIGURI-MATIGARA APEX': {'lat': 26.74426398, 'lng': 88.3537118},
    'M10-MUMBAI SAKINAKA APEX': {'lat': 19.102839, 'lng': 72.882515},
    'N05-DELHI SAMALKHA APEX': {'lat': 28.53231663, 'lng': 77.09848338},
    'O06-COCHIN APEX': {'lat': 10.06337, 'lng': 76.28998},
    'P01-PUNE APEX': {'lat': 18.628523, 'lng': 73.873561},
    'Q05-JAIPUR APEX': {'lat': 26.88274, 'lng': 75.740257},
    'R01-NAGPUR APEX': {'lat': 21.143067, 'lng': 79.007626},
    'T01-PATNA APEX': {'lat': 25.5762215, 'lng': 85.2358959},
    'T02-RANCHI APEX': {'lat': 23.35235, 'lng': 85.377875},
    'U04-VARANASI APEX': {'lat': 25.248513, 'lng': 82.927774},
    'U11-GHAZIABAD APEX': {'lat': 28.68924002, 'lng': 77.43653512},
    'U43-LUCKNOW APEX': {'lat': 26.722948, 'lng': 80.902664},
    'V02-VISHAKAPATNAM APEX': {'lat': 17.73699, 'lng': 83.26624},
    'V06-VIJAYAWADA APEX': {'lat': 16.5479875, 'lng': 80.6892874},
    'W53-BHUBANESWAR AIR APEX': {'lat': 20.302516, 'lng': 85.85346},
    'X01-DHARAPUR APEX': {'lat': 26.13382, 'lng': 91.62004},
    'X21-DIBRUGARH APEX': {'lat': 27.48061152, 'lng': 94.90665145},
    'X03-AGARTALA APEX': {'lat': 23.83510848, 'lng': 91.27031132}
}

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees) using Haversine formula
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Radius of earth in kilometers
    r = 6371

    return round(c * r, 2)

def calculate_duration(distance_km, avg_speed_kmh=50):
    """
    Calculate estimated travel duration
    Default average speed: 50 km/h (typical for highway driving in India)
    """
    hours = distance_km / avg_speed_kmh
    hours_int = int(hours)
    minutes = int((hours - hours_int) * 60)
    return f"{hours_int}h {minutes}m"

def generate_distance_matrix():
    """
    Generate distance matrix between all hubs and airports
    """
    results = []

    print("Calculating distances between hubs and airports...")
    total_calculations = len(hubs) * len(airports)
    current_calc = 0

    for hub_name, hub_coords in hubs.items():
        for airport_city, airport_coords in airports.items():
            current_calc += 1

            # Calculate distance
            distance = calculate_distance(
                hub_coords['lat'], hub_coords['lng'],
                airport_coords['lat'], airport_coords['lng']
            )

            # Calculate duration
            duration = calculate_duration(distance)

            # Store result
            results.append({
                'Hub_Name': hub_name,
                'Hub_Latitude': hub_coords['lat'],
                'Hub_Longitude': hub_coords['lng'],
                'Airport_City': airport_city,
                'Airport_IATA': airport_coords['iata'],
                'Airport_Latitude': airport_coords['lat'],
                'Airport_Longitude': airport_coords['lng'],
                'Distance_KM': distance,
                'Estimated_Duration': duration
            })

            # Progress indicator
            if current_calc % 50 == 0:
                print(f"Progress: {current_calc}/{total_calculations} calculations completed")

    print(f"Completed all {total_calculations} calculations!")
    return results

def save_to_csv(results, filename='hub_airport_distances.csv'):
    """
    Save results to CSV file
    """
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

    # Display summary statistics
    print(f"\nSummary:")
    print(f"Total hub-airport combinations: {len(results)}")
    print(f"Average distance: {df['Distance_KM'].mean():.2f} km")
    print(f"Minimum distance: {df['Distance_KM'].min():.2f} km")
    print(f"Maximum distance: {df['Distance_KM'].max():.2f} km")

    # Show sample data
    print(f"\nFirst 5 rows:")
    print(df.head().to_string(index=False))

def main():
    """
    Main function to run the distance calculation
    """
    print("Hub-Airport Distance Calculator")
    print("=" * 40)
    print(f"Hubs: {len(hubs)}")
    print(f"Airports: {len(airports)}")
    print(f"Total calculations: {len(hubs) * len(airports)}")
    print("=" * 40)

    # Generate distance matrix
    results = generate_distance_matrix()

    # Save to CSV
    save_to_csv(results)

    print("\nDistance calculation completed successfully!")
    print("Note: Distances are straight-line distances using Haversine formula.")
    print("Durations are estimated based on 50 km/h average speed.")

if __name__ == "__main__":
    main()

Hub-Airport Distance Calculator
Hubs: 30
Airports: 26
Total calculations: 780
Calculating distances between hubs and airports...
Progress: 50/780 calculations completed
Progress: 100/780 calculations completed
Progress: 150/780 calculations completed
Progress: 200/780 calculations completed
Progress: 250/780 calculations completed
Progress: 300/780 calculations completed
Progress: 350/780 calculations completed
Progress: 400/780 calculations completed
Progress: 450/780 calculations completed
Progress: 500/780 calculations completed
Progress: 550/780 calculations completed
Progress: 600/780 calculations completed
Progress: 650/780 calculations completed
Progress: 700/780 calculations completed
Progress: 750/780 calculations completed
Completed all 780 calculations!
Results saved to hub_airport_distances.csv

Summary:
Total hub-airport combinations: 780
Average distance: 1133.23 km
Minimum distance: 2.31 km
Maximum distance: 2755.92 km

First 5 rows:
          Hub_Name  Hub_Latitude  Hub

In [ ]:
df = pd.read_csv('hub_airport_distances.csv')

In [ ]:
df.head()

,Hub_Name,Hub_Latitude,Hub_Longitude,Airport_City,Airport_IATA,Airport_Latitude,Airport_Longitude,Distance_KM,Estimated_Duration
0,A01-AHMEDABAD APEX,23.106556,72.683222,AGARTALA,IXA,23.8870,91.2403,1893.00,37h 51m
1,A01-AHMEDABAD APEX,23.106556,72.683222,AHMEDABAD,AMD,23.0726,72.6177,7.69,0h 9m
2,A01-AHMEDABAD APEX,23.106556,72.683222,BANGALORE,BLR,13.1986,77.7066,1222.52,24h 27m
3,A01-AHMEDABAD APEX,23.106556,72.683222,BHUBANESHWAR,BBI,20.2444,85.8178,1393.44,27h 52m
4,A01-AHMEDABAD APEX,23.106556,72.683222,CHANDIGARH,IXC,30.6735,76.7884,934.49,18h 41m


In [ ]:
df = df.drop(['Hub_Latitude', 'Hub_Longitude', 'Airport_IATA', 'Airport_Latitude', 'Airport_Longitude'], axis=1)

In [ ]:
df.head()

,Hub_Name,Airport_City,Distance_KM,Estimated_Duration
0,A01-AHMEDABAD APEX,AGARTALA,1893.00,37h 51m
1,A01-AHMEDABAD APEX,AHMEDABAD,7.69,0h 9m
2,A01-AHMEDABAD APEX,BANGALORE,1222.52,24h 27m
3,A01-AHMEDABAD APEX,BHUBANESHWAR,1393.44,27h 52m
4,A01-AHMEDABAD APEX,CHANDIGARH,934.49,18h 41m


In [ ]:
df.to_csv('hub_to_airport_matrix.csv')

## CASE - 2 logic + Code

In [ ]:
import pandas as pd
import numpy as np

class Case2RouteOptimizer:
    def __init__(self, shipment_data, airport_volume, flight_time_matrix,
                 branch_hub_time, hub_to_airport_time, hub_to_hub_time):
        self.shipment_data = shipment_data
        self.airport_volume = airport_volume
        self.flight_time_matrix = flight_time_matrix
        self.branch_hub_time = branch_hub_time
        self.hub_to_airport_time = hub_to_airport_time
        self.hub_to_hub_time = hub_to_hub_time

        self._build_lookups()

    def _build_lookups(self):
        self.branch_hub_lookup = {(r['Branch'], r['Hub']): r['Duration_hr']
                                  for _, r in self.branch_hub_time.dropna().iterrows()}
        self.hub_airport_lookup = {(r['Hub'], r['Final_Airport_City']): r['Duration_hr']
                                   for _, r in self.hub_to_airport_time.dropna().iterrows()}
        self.hub_hub_lookup = {(r['Hub1'], r['Hub2']): r['Duration_hr']
                               for _, r in self.hub_to_hub_time.dropna().iterrows()}
        self.flight_time_lookup = {(r['Origin'], r['Destination']): r['Estimated_Flight_Time_Minutes'] / 60
                                   for _, r in self.flight_time_matrix.dropna().iterrows()}
        self.airport_volume_lookup = {(r['Origin_Airport'], r['Destination_Airport']): r['Volume_kg']
                                      for _, r in self.airport_volume.dropna().iterrows()}

        self.airport_to_nearest_hub = {}
        for airport in self.hub_to_airport_time['Final_Airport_City'].unique():
            subset = self.hub_to_airport_time[self.hub_to_airport_time['Final_Airport_City'] == airport]
            subset = subset.dropna(subset=['Duration_hr'])
            if not subset.empty:
                best = subset.loc[subset['Duration_hr'].idxmin()]
                self.airport_to_nearest_hub[airport] = (best['Hub'], best['Duration_hr'])

    def compute_case2_route(self, row):
        origin_branch = row['Origin Branch']
        origin_hub = row['Origin Hub']
        dest_branch = row['Destination Branch']
        dest_hub = row['Destination Hub']
        product_type = row['New Product GCP']
        weight = row['Total Weight']

        # Handle if branch = hub
        t1 = 0 if origin_branch == origin_hub else self.branch_hub_lookup.get((origin_branch, origin_hub))
        t13 = 0 if dest_branch == dest_hub else self.branch_hub_lookup.get((dest_branch, dest_hub))

        if None in [t1, t13]:
            return pd.Series([None] * 8)

        best_time, best_path, best_volume, best_detail = float('inf'), None, 0, ""

        for (alt_hub_key, t_hub_hub) in self.hub_hub_lookup.items():
            if alt_hub_key[0] != origin_hub:
                continue

            alt_hub = alt_hub_key[1]

            for (hub, airport), t3 in self.hub_airport_lookup.items():
                if hub != alt_hub:
                    continue

                for (origin_airport, dest_airport), volume in self.airport_volume_lookup.items():
                    if origin_airport != airport or volume < 100:
                        continue

                    t5 = self.flight_time_lookup.get((origin_airport, dest_airport))
                    if t5 is None:
                        continue

                    middle_hub, t7 = self.airport_to_nearest_hub.get(dest_airport, (None, None))
                    if middle_hub is None or t7 is None:
                        continue

                    t9 = self.hub_hub_lookup.get((middle_hub, dest_hub))
                    if t9 is None:
                        continue

                    # Fixed processing times
                    t2, t4, t6, t8, t10 = 2.5, 2.0, 2.0, 2.5, 2.5

                    # Final safety check
                    required_times = [t1, t2, t_hub_hub, t4, t3, t6, t5, t7, t8, t9, t10, t13]
                    if any(t is None for t in required_times):
                        continue

                    total_time = sum(required_times)
                    total_volume = volume + weight

                    if total_volume >= 100 and total_time < best_time:
                        best_time = total_time
                        best_volume = total_volume
                        best_path = (
                            f"{origin_branch} → {origin_hub} → {alt_hub} → {origin_airport} ✈️ {dest_airport} → "
                            f"{middle_hub} → {dest_hub} → {dest_branch}"
                        )
                        best_detail = (
                            f"t1:{t1:.2f} t2:{t2} t_hub_hub:{t_hub_hub:.2f} t4:{t4} "
                            f"t3:{t3:.2f} t6:{t6} t5:{t5:.2f} t7:{t7:.2f} "
                            f"t8:{t8} t9:{t9:.2f} t10:{t10} t13:{t13:.2f}"
                        )

        if best_path:
            return pd.Series([
                product_type, weight, best_path, best_time, best_volume,
                best_detail, "Road → Hub → Airport → ✈️ → Road", "Air Route"
            ])
        else:
            return pd.Series([None] * 8)

    def run_case2_optimizer(self):
        required = self.shipment_data[self.shipment_data['Path_Required'] == 'YES'].copy()
        output = required.apply(self.compute_case2_route, axis=1)
        output.columns = [
            "Product Type", "Weight (kg)", "Suggested Route", "Total Time",
            "Total Volume", "Time_Components", "Route Type", "Recommendation"
        ]
        output.insert(0, "Origin Branch", required["Origin Branch"])
        output.insert(1, "Destination Branch", required["Destination Branch"])
        return output.dropna(subset=["Suggested Route"])

In [ ]:
import pandas as pd

def load_and_run_case2_optimizer():
    # ✅ Load all required CSVs
    shipment_data = pd.read_csv("/content/Daily_datafor_(EP+BP)_ES_path_required_(with_new_rows).csv")
    airport_volume = pd.read_csv('/content/Total_volume_from_airport_to_air(corrected).csv')
    flight_time_matrix = pd.read_csv("/content/acutal_flight_time_matrix.csv")
    branch_hub_time = pd.read_csv("/content/branch_hub_time_haversine_correct.csv")
    hub_to_airport_time = pd.read_csv("/content/Hub_to_airport_time_matrix - Sheet1.csv")
    hub_to_hub_time = pd.read_csv("/content/hub_distance_time_matrix_haversine.csv")

    # ✅ Initialize Case 2 Optimizer
    case2_optimizer = Case2RouteOptimizer(
        shipment_data,
        airport_volume,
        flight_time_matrix,
        branch_hub_time,
        hub_to_airport_time,
        hub_to_hub_time
    )

    # ✅ Run the optimizer and save output
    output_df = case2_optimizer.run_case2_optimizer()

    output_df.to_csv("case2_indirect_routes_1.csv", index=False)
    print("✅ Case 2 route optimization complete and saved.")
    return output_df

# 🔁 Run when this file is executed
if __name__ == "__main__":
    result_df = load_and_run_case2_optimizer()
    print(result_df.head())

✅ Case 2 route optimization complete and saved.
        Origin Branch     Destination Branch Product Type  Weight (kg)  \
0  A01-AHMEDABAD APEX        A05-VAPI BRANCH        EP_BP          0.0   
1  A01-AHMEDABAD APEX  A07-ANKLESHWAR BRANCH        EP_BP          0.0   
2  A01-AHMEDABAD APEX     A11-KHOKHRA BRANCH        EP_BP          0.0   
3  A01-AHMEDABAD APEX    A17-BARODA SCS APEX        EP_BP          0.0   
4  A01-AHMEDABAD APEX     A18-MEHSANA BRANCH        EP_BP          0.0   

                                     Suggested Route  Total Time  \
0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → A04-...   23.919342   
1  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → A04-...   23.079342   
2  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → A04-...   24.709342   
3  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → A04-...   26.489342   
4  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → A04-...   25.719342   

   Total Volume                                    Time_Components  \
0     260.33984  t1:0.00 t2:

## Case 3 final code

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

class Case3RouteOptimizer:
    def __init__(self, shipment_data, airport_volume, flight_time_matrix,
                 branch_hub_time, hub_to_airport_time, hub_to_hub_time):
        self.shipment_data = shipment_data
        self.airport_volume = airport_volume
        self.flight_time_matrix = flight_time_matrix
        self.branch_hub_time = branch_hub_time
        self.hub_to_airport_time = hub_to_airport_time
        self.hub_to_hub_time = hub_to_hub_time
        self.skipped_logs = []

        self._build_optimized_lookups()

    def _build_optimized_lookups(self):
        # Build lookups with null checking done once
        self.branch_hub_lookup = {}
        for _, r in self.branch_hub_time.dropna(subset=['Branch', 'Hub', 'Duration_hr']).iterrows():
            self.branch_hub_lookup[(r['Branch'], r['Hub'])] = r['Duration_hr']

        self.hub_airport_lookup = {}
        for _, r in self.hub_to_airport_time.dropna(subset=['Hub', 'Final_Airport_City', 'Duration_hr']).iterrows():
            self.hub_airport_lookup[(r['Hub'], r['Final_Airport_City'])] = r['Duration_hr']

        self.hub_hub_lookup = {}
        for _, r in self.hub_to_hub_time.dropna(subset=['Hub1', 'Hub2', 'Duration_hr']).iterrows():
            self.hub_hub_lookup[(r['Hub1'], r['Hub2'])] = r['Duration_hr']

        self.flight_time_lookup = {}
        for _, r in self.flight_time_matrix.dropna(subset=['Origin', 'Destination', 'Estimated_Flight_Time_Minutes']).iterrows():
            self.flight_time_lookup[(r['Origin'], r['Destination'])] = r['Estimated_Flight_Time_Minutes'] / 60

        self.volume_lookup = {}
        for _, r in self.airport_volume.dropna(subset=['Origin_Airport', 'Destination_Airport', 'Volume_kg']).iterrows():
            self.volume_lookup[(r['Origin_Airport'], r['Destination_Airport'])] = r['Volume_kg']

        # Pre-compute airport to hub mapping
        self.airport_to_hub = {}
        airport_hub_data = self.hub_to_airport_time.dropna(subset=['Final_Airport_City', 'Hub', 'Duration_hr'])
        for airport in airport_hub_data['Final_Airport_City'].unique():
            subset = airport_hub_data[airport_hub_data['Final_Airport_City'] == airport]
            if not subset.empty:
                best_idx = subset['Duration_hr'].idxmin()
                best = subset.loc[best_idx]
                self.airport_to_hub[airport] = (best['Hub'], best['Duration_hr'])

        # Pre-compute hub connections for faster lookup
        self.hub_connections = defaultdict(list)
        for (h1, h2), duration in self.hub_hub_lookup.items():
            self.hub_connections[h1].append((h2, duration))

        # Pre-compute available airports per hub for faster iteration
        self.hub_airports = defaultdict(list)
        for (hub, airport), duration in self.hub_airport_lookup.items():
            self.hub_airports[hub].append((airport, duration))

        # Cache unique airports to avoid repeated unique() calls
        self.unique_airports = set(self.hub_to_airport_time['Final_Airport_City'].dropna().unique())

        # Pre-compute constants
        self.constants = {
            't2_5': 2.5, 't4_5': 2.0, 't5_5': 2.0, 't7': 2.5, 't8': 2.5
        }
    def compute_case3_path(self, row):
        origin_branch = row['Origin Branch']
        origin_hub = row['Origin Hub']
        dest_branch = row['Destination Branch']
        dest_hub = row['Destination Hub']
        dest_airport = row['Destination Airport']
        product_type = row['New Product GCP']
        weight = row['Total Weight']

        # ✅ Handle branch == hub (direct locations)
        t1 = 0 if origin_branch == origin_hub else self.branch_hub_lookup.get((origin_branch, origin_hub))
        t13 = 0 if dest_branch == dest_hub else self.branch_hub_lookup.get((dest_branch, dest_hub))
        if t1 is None or t13 is None:
            self.skipped_logs.append((origin_branch, dest_branch, "Missing branch-hub time"))
            return pd.Series([None] * 8)

        # Get destination airport's nearest hub and t5
        dest_airport_hub_info = self.airport_to_hub.get(dest_airport)
        if dest_airport_hub_info is None:
            self.skipped_logs.append((origin_branch, dest_branch, "No hub found for destination airport"))
            return pd.Series([None] * 8)
        dest_middle_hub, t5 = dest_airport_hub_info

        # ✅ Calculate t6 smartly: if same hub, 0; else find time
        if dest_middle_hub == dest_hub:
            t6 = 0
        elif (dest_branch, dest_middle_hub) in self.branch_hub_lookup:
            t6 = self.branch_hub_lookup[(dest_branch, dest_middle_hub)]
        elif (dest_middle_hub, dest_hub) in self.hub_hub_lookup:
            t6 = self.hub_hub_lookup[(dest_middle_hub, dest_hub)]
        else:
            self.skipped_logs.append((origin_branch, dest_branch, "No valid t6 from airport hub to destination hub"))
            return pd.Series([None] * 8)

        best_time, best_path, best_volume, best_components = float('inf'), "", 0, ""
        t2_5, t4_5, t5_5, t7, t8 = (self.constants[k] for k in ['t2_5', 't4_5', 't5_5', 't7', 't8'])

        for other_hub, t2 in self.hub_connections[origin_hub]:
            for airport, t3 in self.hub_airports[other_hub]:
                t4 = self.flight_time_lookup.get((airport, dest_airport))
                if t4 is None:
                    continue

                volume = self.volume_lookup.get((airport, dest_airport), 0)
                total_volume = volume + weight
                if total_volume < 100:
                    continue

                # Ensure all segments are valid
                segment_times = [t1, t2_5, t2, t7, t3, t4_5, t4, t5_5, t5, t8, t6, t13]
                if any(t is None for t in segment_times):
                    self.skipped_logs.append((origin_branch, dest_branch, "Incomplete path timing"))
                    continue

                total_time = sum(segment_times)

                if total_time < best_time:
                    best_time = total_time
                    best_volume = total_volume
                    best_path = (
                        f"{origin_branch} → {origin_hub} → {other_hub} → {airport} ✈️ {dest_airport} → "
                        f"{dest_middle_hub} → {dest_hub} → {dest_branch}"
                    )
                    best_components = (
                        f"t1:{t1:.2f} t2:{t2:.2f} t3:{t3:.2f} t4:{t4:.2f} t5:{t5:.2f} t6:{t6:.2f} "
                        f"t2.5:{t2_5} t4.5:{t4_5} t5.5:{t5_5} t7:{t7} t8:{t8} t13:{t13:.2f}"
                    )

        if best_path:
            return pd.Series([
                product_type, weight, best_path, best_time, best_volume,
                best_components, "Road + Road + Air + Road", "Air Route"
            ])
        else:
            self.skipped_logs.append((origin_branch, dest_branch, "No valid Case 3 route found"))
            return pd.Series([None] * 8)


    def run_case3_optimizer(self):
        print("🔄 Running Case 3 optimizer...")

        # Filter once and copy
        required = self.shipment_data[self.shipment_data["Path_Required"] == "YES"].copy()

        if required.empty:
            print("No shipments require path optimization")
            return pd.DataFrame(), pd.DataFrame(columns=["Origin Branch", "Destination Branch", "Skip Reason"])

        # Apply optimization
        output = required.apply(self.compute_case3_path, axis=1)

        # Set column names
        output.columns = [
            "Product Type", "Weight (kg)", "Suggested Route", "Indirect Total Time",
            "Total Volume", "Time_Components", "Route Type", "Recommendation"
        ]

        # Insert origin/destination columns
        output.insert(0, "Origin Branch", required["Origin Branch"])
        output.insert(1, "Destination Branch", required["Destination Branch"])

        # Create skipped DataFrame
        skipped_df = pd.DataFrame(self.skipped_logs, columns=["Origin Branch", "Destination Branch", "Skip Reason"])

        # Filter valid routes
        valid_output = output.dropna(subset=["Suggested Route"])

        return valid_output, skipped_df


# === File runner ===
def load_and_run_case3_optimizer():
    shipment_file = "/content/Daily_datafor_(EP+BP)_ES_path_required_(with_new_rows).csv"
    flight_time_file = "/content/acutal_flight_time_matrix.csv"
    branch_hub_time_file = "/content/branch_hub_time_haversine_correct.csv"
    hub_to_airport_time_file = "/content/Hub_to_airport_time_matrix - Sheet1.csv"
    hub_hub_time_file = "/content/hub_distance_time_matrix_haversine.csv"

    # Load data
    print("📂 Loading data files...")
    shipment_data = pd.read_csv(shipment_file)
    airport_volume = pd.read_csv('/content/Total_volume_from_airport_to_air(corrected).csv')  # Using the existing volume_df variable
    flight_time_matrix = pd.read_csv(flight_time_file)
    branch_hub_time = pd.read_csv(branch_hub_time_file)
    hub_to_airport_time = pd.read_csv(hub_to_airport_time_file)
    hub_to_hub_time = pd.read_csv(hub_hub_time_file)

    print("🚀 Initializing optimizer...")
    optimizer = Case3RouteOptimizer(
        shipment_data, airport_volume, flight_time_matrix,
        branch_hub_time, hub_to_airport_time, hub_to_hub_time
    )

    print("⚡ Running optimization...")
    final_output, skipped_df = optimizer.run_case3_optimizer()

    print("💾 Saving results...")
    final_output.to_csv("case3_routes_output_new2.csv", index=False)
    skipped_df.to_csv("case3_skipped_od_pairs1.csv", index=False)

    print(f"✅ Saved {len(final_output)} routes and {len(skipped_df)} skipped pairs")
    if not final_output.empty:
        print("\n📋 Sample results:")
        print(final_output.head(3))
    else:
        print("⚠️ No valid routes found")


if __name__ == "__main__":
    load_and_run_case3_optimizer()

📂 Loading data files...
🚀 Initializing optimizer...
⚡ Running optimization...
🔄 Running Case 3 optimizer...
💾 Saving results...
✅ Saved 234394 routes and 20253 skipped pairs

📋 Sample results:
        Origin Branch     Destination Branch Product Type  Weight (kg)  \
0  A01-AHMEDABAD APEX        A05-VAPI BRANCH        EP_BP          0.0   
1  A01-AHMEDABAD APEX  A07-ANKLESHWAR BRANCH        EP_BP          0.0   
2  A01-AHMEDABAD APEX     A11-KHOKHRA BRANCH        EP_BP          0.0   

                                     Suggested Route  Indirect Total Time  \
0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → P01-...            22.909342   
1  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → P01-...            22.069342   
2  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → P01-...            20.773311   

   Total Volume                                    Time_Components  \
0      132.2208  t1:0.00 t2:7.33 t3:0.24 t4:1.25 t5:0.77 t6:0.0...   
1      132.2208  t1:0.00 t2:7.33 t3:0.24 t4:1.25 t5:0.77 t6:0

## Case 4 : Evaluating direct connections also

In [ ]:
import pandas as pd

class Case4DirectAirOptimizer:
    def __init__(self, shipment_data, airport_volume, flight_time_matrix,
                 branch_hub_time, hub_to_airport_time):
        self.shipment_data = shipment_data
        self.airport_volume = airport_volume
        self.flight_time_matrix = flight_time_matrix
        self.branch_hub_time = branch_hub_time
        self.hub_to_airport_time = hub_to_airport_time

        self.skipped_logs = []
        self.evaluated_airport_pairs = []

        self._build_lookups()

    def _build_lookups(self):
        self.branch_hub_lookup = {
            (r['Branch'], r['Hub']): r['Duration_hr']
            for _, r in self.branch_hub_time.dropna().iterrows()
        }
        self.hub_airport_lookup = {
            (r['Hub'], r['Final_Airport_City']): r['Duration_hr']
            for _, r in self.hub_to_airport_time.dropna().iterrows()
        }
        self.flight_time_lookup = {
            (r['Origin'], r['Destination']): r['Estimated_Flight_Time_Minutes'] / 60
            for _, r in self.flight_time_matrix.dropna().iterrows()
        }
        self.volume_lookup = {
            (r['Origin_Airport'], r['Destination_Airport']): r['Volume_kg']
            for _, r in self.airport_volume.dropna().iterrows()
        }
        self.constants = {
            'in_cooling': 2.0,
            'out_cooling': 2.0,
            'origin_hub_processing': 2.5,
            'dest_hub_processing': 2.5
        }

    def safe_lookup(self, lookup_dict, key, default=0.0):
        """Lookup from dict; return 0 if key not found or value is NaN."""
        val = lookup_dict.get(key)
        return default if val is None or pd.isna(val) else val

    def compute_case4_route(self, row):
        origin_branch = row['Origin Branch']
        origin_hub = row['Origin Hub']
        origin_airport = row['Origin Airport']
        dest_airport = row['Destination Airport']
        dest_hub = row['Destination Hub']
        dest_branch = row['Destination Branch']
        product_type = row['New Product GCP']
        weight = row['Total Weight']

        # Core times
        t1 = 0.0 if origin_branch == origin_hub else self.safe_lookup(self.branch_hub_lookup, (origin_branch, origin_hub))
        t2 = self.safe_lookup(self.hub_airport_lookup, (origin_hub, origin_airport))
        t6 = self.safe_lookup(self.hub_airport_lookup, (dest_hub, dest_airport))
        t7 = 0.0 if dest_branch == dest_hub else self.safe_lookup(self.branch_hub_lookup, (dest_branch, dest_hub))

        # Required: flight and volume check
        t4 = self.flight_time_lookup.get((origin_airport, dest_airport))
        self.evaluated_airport_pairs.append((origin_airport, dest_airport))
        if t4 is None or pd.isna(t4):
            self.skipped_logs.append((origin_branch, dest_branch, "Missing flight time"))
            return pd.Series([None] * 8)

        volume = self.volume_lookup.get((origin_airport, dest_airport), 0)
        total_volume = volume + weight

        if total_volume < 100:
            self.skipped_logs.append((origin_branch, dest_branch, f"Insufficient volume: {total_volume:.2f}"))
            return pd.Series([None] * 8)

        # Fixed times
        t3 = self.constants['in_cooling']
        t5 = self.constants['out_cooling']
        t1_5 = self.constants['origin_hub_processing']
        t7_5 = self.constants['dest_hub_processing']

        total_time = t1 + t1_5 + t2 + t3 + t4 + t5 + t6 + t7_5 + t7

        suggested_path = (
            f"{origin_branch} → {origin_hub} → {origin_airport} ✈️ {dest_airport} → "
            f"{dest_hub} → {dest_branch}"
        )
        time_breakdown = (
            f"t1:{t1:.2f} t1.5:{t1_5} t2:{t2:.2f} t3:{t3} t4:{t4:.2f} "
            f"t5:{t5} t6:{t6:.2f} t7.5:{t7_5} t7:{t7:.2f}"
        )

        return pd.Series([
            product_type, weight, suggested_path, total_time, total_volume,
            time_breakdown, "Direct Air Path", "Air Route"
        ])

    def run_case4_optimizer(self):
        required = self.shipment_data[self.shipment_data["Path_Required"] == "YES"].copy()
        if required.empty:
            print("❌ No records found with Path_Required = YES")
            return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

        output = required.apply(self.compute_case4_route, axis=1)
        output.columns = [
            "Product Type", "Weight (kg)", "Suggested Route", "Total Time",
            "Total Volume", "Time_Components", "Route Type", "Recommendation"
        ]
        output.insert(0, "Origin Branch", required["Origin Branch"])
        output.insert(1, "Destination Branch", required["Destination Branch"])

        valid = output.dropna(subset=["Suggested Route"])
        skipped = pd.DataFrame(self.skipped_logs, columns=["Origin Branch", "Destination Branch", "Skip Reason"])
        eval_airports = pd.DataFrame(self.evaluated_airport_pairs, columns=["Origin Airport", "Destination Airport"])
        return valid, skipped, eval_airports

In [ ]:
def load_and_run_case4_optimizer():
    shipment_data = pd.read_csv("/content/Daily_datafor_(EP+BP)_ES_path_required_(with_new_rows).csv")
    airport_volume = pd.read_csv("/content/Total_volume_from_airport_to_air(corrected).csv")
    flight_time_matrix = pd.read_csv("/content/acutal_flight_time_matrix.csv")
    branch_hub_time = pd.read_csv("/content/branch_hub_time_haversine_correct.csv")
    hub_to_airport_time = pd.read_csv("/content/Hub_to_airport_time_matrix - Sheet1.csv")

    optimizer = Case4DirectAirOptimizer(
        shipment_data,
        airport_volume,
        flight_time_matrix,
        branch_hub_time,
        hub_to_airport_time
    )

    valid, skipped, evaluated_pairs = optimizer.run_case4_optimizer()

    valid.to_csv("Case4_direct_air_routes_output.csv", index=False)
    skipped.to_csv("Case4_skipped_reasons.csv", index=False)
    evaluated_pairs.drop_duplicates().to_csv("Case4_evaluated_airport_pairs.csv", index=False)

    print(f"✅ Case 4 complete: {len(valid)} routes, {len(skipped)} skipped.")
    return valid, skipped, evaluated_pairs

# Run
case4_result, case4_skipped, case4_eval_pairs = load_and_run_case4_optimizer()
print(case4_result.head())

✅ Case 4 complete: 150476 routes, 104171 skipped.
         Origin Branch              Destination Branch Product Type  \
17  A01-AHMEDABAD APEX  B01-MANGALORE SUPER FRANCHISEE           ES   
18  A01-AHMEDABAD APEX               B02-MYSORE BRANCH           ES   
19  A01-AHMEDABAD APEX         B06-BOMMANAHALLI BRANCH           ES   
20  A01-AHMEDABAD APEX    B10-BANGALORE YELAHANKA APEX           ES   
21  A01-AHMEDABAD APEX    B16-BELGAUM SUPER FRANCHISEE           ES   

    Weight (kg)                                    Suggested Route  \
17          0.0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...   
18          0.0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...   
19          0.0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...   
20          0.0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...   
21          0.0  A01-AHMEDABAD APEX → A01-AHMEDABAD APEX → AHME...   

    Total Time  Total Volume  \
17   18.099624     882.09540   
18   14.879624     882.09540   
19   1

## Road time calculation

In [ ]:
import pandas as pd

def compute_full_road_time(shipments_file, branch_hub_file, hub_hub_file, output_file="road_time_output.csv"):
    print("📥 Reading input files...")

    df = pd.read_csv(shipments_file)
    branch_hub_df = pd.read_csv(branch_hub_file)
    hub_hub_df = pd.read_csv(hub_hub_file)

    print("⚙️ Building lookup dictionaries...")

    branch_hub_lookup = {
        (row['Branch'], row['Hub']): row['Duration_hr']
        for _, row in branch_hub_df.dropna(subset=['Branch', 'Hub', 'Duration_hr']).iterrows()
    }

    hub_hub_lookup = {
        (row['Hub1'], row['Hub2']): row['Duration_hr']
        for _, row in hub_hub_df.dropna(subset=['Hub1', 'Hub2', 'Duration_hr']).iterrows()
    }

    full_road_times = []

    print("🚛 Computing full road times...")

    for idx, row in df.iterrows():
        ob = row['Origin Branch']
        oh = row['Origin Hub']
        dh = row['Destination Hub']
        db = row['Destination Branch']

        # Origin Branch → Origin Hub
        t1 = 0 if ob == oh else branch_hub_lookup.get((ob, oh), None)
        # Origin Hub → Destination Hub
        t2 = hub_hub_lookup.get((oh, dh), None)
        # Destination Hub → Destination Branch
        t3 = 0 if dh == db else branch_hub_lookup.get((db, dh), None)

        # If any required time is missing, mark as None
        if (t1 is None and ob != oh) or (t2 is None) or (t3 is None and db != dh):
            full_road_times.append(None)
            continue

        # Hub processing times
        t_proc1 = 0 if ob == oh else 2.5
        t_proc2 = 0 if dh == db else 2.5

        total_time = t1 + t_proc1 + t2 + t_proc2 + t3
        full_road_times.append(total_time)

    print("📊 Adding to DataFrame and saving...")

    df['Full Road Time'] = full_road_times
    df.to_csv(output_file, index=False)

    print(f"✅ Full road time saved to: {output_file}")
    print(df[['Origin Branch', 'Destination Branch', 'Full Road Time']].head(3))

# === Runner ===
if __name__ == "__main__":
    compute_full_road_time(
        shipments_file="/content/Daily_datafor_(EP+BP)_ES_path_required_updated.csv",
        branch_hub_file="/content/branch_to_hub_time_haversine.csv",
        hub_hub_file="/content/hub_to_hub_time.csv",
        output_file="full_road_time_output.csv"
    )

📥 Reading input files...
⚙️ Building lookup dictionaries...
🚛 Computing full road times...
📊 Adding to DataFrame and saving...
✅ Full road time saved to: full_road_time_output.csv
        Origin Branch  Destination Branch  Full Road Time
0  A01-AHMEDABAD APEX  A01-AHMEDABAD APEX             NaN
1  A01-AHMEDABAD APEX   A02-BARODA BRANCH             NaN
2  A01-AHMEDABAD APEX   A03-RAJKOT BRANCH             NaN


In [ ]:
df_road_time = pd.read_csv('/content/full_road_time_output.csv')

In [ ]:
df_road_time.head(20)

,Unnamed: 0,Origin Branch,Origin Hub,Origin Airport,Destination Airport,Destination Hub,Destination Branch,New Product GCP,Total Weight,Total_conno,Flight Mode,Path_Required,Full Road Time
0,0,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,AHMEDABAD,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,EP_BP,6.44032,1.96,Road,YES,NaN
1,1,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,AHMEDABAD,A01-AHMEDABAD APEX,A02-BARODA BRANCH,EP_BP,0.24000,0.08,Road,YES,NaN
2,2,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,AHMEDABAD,A01-AHMEDABAD APEX,A03-RAJKOT BRANCH,EP_BP,11.16000,5.12,Road,YES,NaN
3,3,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,AHMEDABAD,A01-AHMEDABAD APEX,A16-KUTCH DDLP SUPER FRANCHISE,EP_BP,0.80000,0.60,Road,YES,NaN
4,4,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,AHMEDABAD,A01-AHMEDABAD APEX,A47-BHUJ SUPER FRANCHISEE,EP_BP,0.24000,0.24,Road,YES,NaN
5,5,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,BANGALORE,B10-BANGALORE YELAHANKA APEX,B01-MANGALORE SUPER FRANCHISEE,EP_BP,0.02200,0.04,PRIME,YES,33.13
6,6,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,BANGALORE,B10-BANGALORE YELAHANKA APEX,B02-MYSORE BRANCH,EP_BP,0.04000,0.04,PRIME,YES,29.91
7,7,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,BANGALORE,B10-BANGALORE YELAHANKA APEX,B06-BOMMANAHALLI BRANCH,EP_BP,0.04000,0.04,PRIME,YES,27.54
8,8,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,BANGALORE,B10-BANGALORE YELAHANKA APEX,B07-CHAMRAJPET BRANCH,EP_BP,0.16000,0.04,PRIME,YES,27.41
9,9,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,AHMEDABAD,BANGALORE,B10-BANGALORE YELAHANKA APEX,B10-BANGALORE YELAHANKA APEX,EP_BP,6.04000,1.32,PRIME,YES,24.56


In [ ]:
# prompt: keep only origin branch destination branch and full road time in a seperate data frame from df_road_time

df_road_subset = df_road_time[['Origin Branch', 'Destination Branch', 'Full Road Time']].copy()

print(df_road_subset.head())


        Origin Branch              Destination Branch  Full Road Time
0  A01-AHMEDABAD APEX              A01-AHMEDABAD APEX             NaN
1  A01-AHMEDABAD APEX               A02-BARODA BRANCH             NaN
2  A01-AHMEDABAD APEX               A03-RAJKOT BRANCH             NaN
3  A01-AHMEDABAD APEX  A16-KUTCH DDLP SUPER FRANCHISE             NaN
4  A01-AHMEDABAD APEX       A47-BHUJ SUPER FRANCHISEE             NaN


In [ ]:
df_road_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288067 entries, 0 to 288066
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Origin Branch       288067 non-null  object 
 1   Destination Branch  288067 non-null  object 
 2   Full Road Time      155453 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.6+ MB


In [ ]:
# prompt: take only unqiue rows in df_road_subset, no need of 2 same rows

df_road_subset = df_road_subset.drop_duplicates().reset_index(drop=True)
print("\nDataFrame after dropping duplicates:")
print(df_road_subset.head())
print("\nDataFrame shape after dropping duplicates:", df_road_subset.shape)



DataFrame after dropping duplicates:
        Origin Branch              Destination Branch  Full Road Time
0  A01-AHMEDABAD APEX              A01-AHMEDABAD APEX             NaN
1  A01-AHMEDABAD APEX               A02-BARODA BRANCH             NaN
2  A01-AHMEDABAD APEX               A03-RAJKOT BRANCH             NaN
3  A01-AHMEDABAD APEX  A16-KUTCH DDLP SUPER FRANCHISE             NaN
4  A01-AHMEDABAD APEX       A47-BHUJ SUPER FRANCHISEE             NaN

DataFrame shape after dropping duplicates: (178040, 3)


In [ ]:
df_road_subset.to_csv('Road_time_between_OD.csv')

In [ ]:
import pandas as pd

def compute_full_road_time_robust(
    shipments_file,
    branch_hub_file,
    hub_hub_file,
    output_file="full_road_time_output.csv"
):
    print("📥 Reading files...")
    df = pd.read_csv(shipments_file)
    branch_hub_df = pd.read_csv(branch_hub_file)
    hub_hub_df = pd.read_csv(hub_hub_file)

    print("⚙️ Building lookups...")
    # Strip whitespaces for consistency
    branch_hub_df['Branch'] = branch_hub_df['Branch'].str.strip()
    branch_hub_df['Hub'] = branch_hub_df['Hub'].str.strip()
    hub_hub_df['Hub1'] = hub_hub_df['Hub1'].str.strip()
    hub_hub_df['Hub2'] = hub_hub_df['Hub2'].str.strip()
    df['Origin Branch'] = df['Origin Branch'].str.strip()
    df['Origin Hub'] = df['Origin Hub'].str.strip()
    df['Destination Branch'] = df['Destination Branch'].str.strip()
    df['Destination Hub'] = df['Destination Hub'].str.strip()

    # Build lookups
    branch_to_hub_time = {
        (row['Branch'], row['Hub']): row['Duration_hr']
        for _, row in branch_hub_df.dropna(subset=['Branch', 'Hub', 'Duration_hr']).iterrows()
    }
    hub_to_hub_time = {
        (row['Hub1'], row['Hub2']): row['Duration_hr']
        for _, row in hub_hub_df.dropna(subset=['Hub1', 'Hub2', 'Duration_hr']).iterrows()
    }

    full_road_times = []
    reasons = []

    print("🚛 Calculating road times...")
    for _, row in df.iterrows():
        ob, oh = row['Origin Branch'], row['Origin Hub']
        db, dh = row['Destination Branch'], row['Destination Hub']

        # t1: Origin Branch → Origin Hub
        t1 = 0 if ob == oh else branch_to_hub_time.get((ob, oh), branch_to_hub_time.get((oh, ob)))

        # t2: Origin Hub → Destination Hub
        t2 = hub_to_hub_time.get((oh, dh), hub_to_hub_time.get((dh, oh)))

        # t3: Destination Hub → Destination Branch
        t3 = 0 if dh == db else branch_to_hub_time.get((db, dh), branch_to_hub_time.get((dh, db)))

        # Processing times
        t_proc1 = 0 if ob == oh else 2.5
        t_proc2 = 0 if dh == db else 2.5

        if t1 is None:
            reasons.append("Missing t1: Origin Branch → Origin Hub")
            full_road_times.append(None)
            continue
        if t2 is None:
            reasons.append("Missing t2: Origin Hub → Destination Hub")
            full_road_times.append(None)
            continue
        if t3 is None:
            reasons.append("Missing t3: Destination Hub → Destination Branch")
            full_road_times.append(None)
            continue

        total_time = t1 + t_proc1 + t2 + t_proc2 + t3
        full_road_times.append(total_time)
        reasons.append("OK")

    df['Full Road Time'] = full_road_times
    df['Road Time Status'] = reasons

    df.to_csv(output_file, index=False)
    print(f"✅ Done. Results saved to {output_file}")
    print(df[['Origin Branch', 'Destination Branch', 'Full Road Time', 'Road Time Status']].head(5))

# === Runner ===
if __name__ == "__main__":
    compute_full_road_time_robust(
        shipments_file="/content/Daily_datafor_(EP+BP)_ES_path_required_updated.csv",
        branch_hub_file="/content/branch_to_hub_time_haversine.csv",
        hub_hub_file="/content/hub_to_hub_time.csv",
        output_file="full_road_time_output.csv"
    )

📥 Reading files...
⚙️ Building lookups...
🚛 Calculating road times...
✅ Done. Results saved to full_road_time_output.csv
        Origin Branch              Destination Branch  Full Road Time  \
0  A01-AHMEDABAD APEX              A01-AHMEDABAD APEX             NaN   
1  A01-AHMEDABAD APEX               A02-BARODA BRANCH             NaN   
2  A01-AHMEDABAD APEX               A03-RAJKOT BRANCH             NaN   
3  A01-AHMEDABAD APEX  A16-KUTCH DDLP SUPER FRANCHISE             NaN   
4  A01-AHMEDABAD APEX       A47-BHUJ SUPER FRANCHISEE             NaN   

                           Road Time Status  
0  Missing t2: Origin Hub → Destination Hub  
1  Missing t2: Origin Hub → Destination Hub  
2  Missing t2: Origin Hub → Destination Hub  
3  Missing t2: Origin Hub → Destination Hub  
4  Missing t2: Origin Hub → Destination Hub  


In [ ]:
df_road = pd.read_csv('/content/full_road_time_output.csv')

In [ ]:
# prompt: keep only origin branch destination branch and full road time in a seperate data frame from df_road_time

df_road_subset = df_road[['Origin Branch', 'Destination Branch', 'Full Road Time', 'Road Time Status']].copy()

print(df_road_subset.head())


        Origin Branch              Destination Branch  Full Road Time  \
0  A01-AHMEDABAD APEX              A01-AHMEDABAD APEX             NaN   
1  A01-AHMEDABAD APEX               A02-BARODA BRANCH             NaN   
2  A01-AHMEDABAD APEX               A03-RAJKOT BRANCH             NaN   
3  A01-AHMEDABAD APEX  A16-KUTCH DDLP SUPER FRANCHISE             NaN   
4  A01-AHMEDABAD APEX       A47-BHUJ SUPER FRANCHISEE             NaN   

                           Road Time Status  
0  Missing t2: Origin Hub → Destination Hub  
1  Missing t2: Origin Hub → Destination Hub  
2  Missing t2: Origin Hub → Destination Hub  
3  Missing t2: Origin Hub → Destination Hub  
4  Missing t2: Origin Hub → Destination Hub  


In [ ]:
# prompt: take only unqiue rows in df_road_subset, no need of 2 same rows

df_road_subset = df_road_subset.drop_duplicates().reset_index(drop=True)
print("\nDataFrame after dropping duplicates:")
print(df_road_subset.head())
print("\nDataFrame shape after dropping duplicates:", df_road_subset.shape)



DataFrame after dropping duplicates:
        Origin Branch              Destination Branch  Full Road Time  \
0  A01-AHMEDABAD APEX              A01-AHMEDABAD APEX             NaN   
1  A01-AHMEDABAD APEX               A02-BARODA BRANCH             NaN   
2  A01-AHMEDABAD APEX               A03-RAJKOT BRANCH             NaN   
3  A01-AHMEDABAD APEX  A16-KUTCH DDLP SUPER FRANCHISE             NaN   
4  A01-AHMEDABAD APEX       A47-BHUJ SUPER FRANCHISEE             NaN   

                           Road Time Status  
0  Missing t2: Origin Hub → Destination Hub  
1  Missing t2: Origin Hub → Destination Hub  
2  Missing t2: Origin Hub → Destination Hub  
3  Missing t2: Origin Hub → Destination Hub  
4  Missing t2: Origin Hub → Destination Hub  

DataFrame shape after dropping duplicates: (178040, 4)


In [ ]:
df_road_subset.to_csv('Correct_road_time.csv')

## Finding Branch-city mapping

In [ ]:
df_branch = pd.read_excel('/content/Branch Master.xlsx')

In [ ]:
df_branch.head()

,Branch,Branch_Name,Branch City,State,Region,Zone,Region Name,City Code,Updated ON,OFFICE TYPE,Mega Region,HUB_CODE,Virtual Branch,Office_ID,OFF_LOCATION
0,A01,AHMEDABAD APEX,AHMEDABAD,GUJARAT,AMD,West,AHMEDABAD,AMD,2021-10-12 00:00:00,HO,AMD,H,N,1,L
1,A02,BARODA BRANCH,BARODA,GUJARAT,AMD,West,AHMEDABAD,BDQ,2022-07-11 00:00:00,BO,AMD,B,N,2,O
2,A03,RAJKOT BRANCH,RAJKOT,GUJARAT,AMD,West,AHMEDABAD,RJT,2021-10-12 00:00:00,BO,AMD,B,N,3,O
3,A04,SURAT APEX,SURAT,GUJARAT,AMD,West,AHMEDABAD,SUT,2021-10-12 00:00:00,HO,AMD,H,N,4,O
4,A05,VAPI BRANCH,VAPI,GUJARAT,AMD,West,AHMEDABAD,VAP,2021-10-12 00:00:00,BO,AMD,B,N,5,O


In [ ]:
# prompt: combine Branch and Branch_Name values as A01 and AHMEDABAD APEX to A01-AHMEDABAD APEX in df_branch

df_branch['Branch_ID_Name'] = df_branch['Branch'] + '-' + df_branch['Branch_Name']
print(df_branch[['Branch', 'Branch_Name', 'Branch_ID_Name']].head())


  Branch     Branch_Name      Branch_ID_Name
0    A01  AHMEDABAD APEX  A01-AHMEDABAD APEX
1    A02   BARODA BRANCH   A02-BARODA BRANCH
2    A03   RAJKOT BRANCH   A03-RAJKOT BRANCH
3    A04      SURAT APEX      A04-SURAT APEX
4    A05     VAPI BRANCH     A05-VAPI BRANCH


In [ ]:
df_branch.head()

,Branch,Branch_Name,Branch City,State,Region,Zone,Region Name,City Code,Updated ON,OFFICE TYPE,Mega Region,HUB_CODE,Virtual Branch,Office_ID,OFF_LOCATION,Branch_ID_Name
0,A01,AHMEDABAD APEX,AHMEDABAD,GUJARAT,AMD,West,AHMEDABAD,AMD,2021-10-12 00:00:00,HO,AMD,H,N,1,L,A01-AHMEDABAD APEX
1,A02,BARODA BRANCH,BARODA,GUJARAT,AMD,West,AHMEDABAD,BDQ,2022-07-11 00:00:00,BO,AMD,B,N,2,O,A02-BARODA BRANCH
2,A03,RAJKOT BRANCH,RAJKOT,GUJARAT,AMD,West,AHMEDABAD,RJT,2021-10-12 00:00:00,BO,AMD,B,N,3,O,A03-RAJKOT BRANCH
3,A04,SURAT APEX,SURAT,GUJARAT,AMD,West,AHMEDABAD,SUT,2021-10-12 00:00:00,HO,AMD,H,N,4,O,A04-SURAT APEX
4,A05,VAPI BRANCH,VAPI,GUJARAT,AMD,West,AHMEDABAD,VAP,2021-10-12 00:00:00,BO,AMD,B,N,5,O,A05-VAPI BRANCH


In [ ]:
# prompt: keep only Branch_ID_Name and Branch City and drop rest of the columns

df_branch_mapping = df_branch[['Branch_ID_Name', 'Branch City']].copy()

print(df_branch_mapping.head())


       Branch_ID_Name Branch City
0  A01-AHMEDABAD APEX   AHMEDABAD
1   A02-BARODA BRANCH      BARODA
2   A03-RAJKOT BRANCH      RAJKOT
3      A04-SURAT APEX       SURAT
4     A05-VAPI BRANCH        VAPI


In [ ]:
# prompt: check if below Branch_ID_Name exist in df_branch_mapping
# J26-KUNDLI BRANCH
# U143-GOMTI NAGAR BRANCH
# R11-NAGPUR PANDE LAYOUT BRANCH
# R32-NAGPUR GHAT ROAD BRANCH
# P24-WAGHOLI BRANCH
# A44-ANAND BRANCH
# I41-MANDIDEEP SERVICE CENTER
# K29-DIAMOND HARBOR SORTING OFFICE
# T40-HAZIPUR SORTING OFFICE
# T89-SAGUNA MORE BRANCH
# U142-BASTI BRANCH

branch_names = [
    "J26-KUNDLI BRANCH",
    "U143-GOMTI NAGAR BRANCH",
    "R11-NAGPUR PANDE LAYOUT BRANCH",
    "R32-NAGPUR GHAT ROAD BRANCH",
    "P24-WAGHOLI BRANCH",
    "A44-ANAND BRANCH",
    "I41-MANDIDEEP SERVICE CENTER",
    "K29-DIAMOND HARBOR SORTING OFFICE",
    "T40-HAZIPUR SORTING OFFICE",
    "T89-SAGUNA MORE BRANCH",
    "U142-BASTI BRANCH"
]

# Check if each branch name exists in the df_branch_mapping DataFrame
check_results = df_branch_mapping['Branch_ID_Name'].isin(branch_names)

print("Check results for each branch name:")
for name, exists in zip(branch_names, check_results):
    print(f"{name}: {'Exists' if exists else 'Does Not Exist'}")

# Optionally, print the branches from your list that are NOT found
not_found = [name for name in branch_names if name not in df_branch_mapping['Branch_ID_Name'].values]
if not_found:
    print("\nBranch names from the list NOT found in df_branch_mapping:")
    for name in not_found:
        print(name)
else:
    print("\nAll specified branch names were found in df_branch_mapping.")

Check results for each branch name:
J26-KUNDLI BRANCH: Does Not Exist
U143-GOMTI NAGAR BRANCH: Does Not Exist
R11-NAGPUR PANDE LAYOUT BRANCH: Does Not Exist
R32-NAGPUR GHAT ROAD BRANCH: Does Not Exist
P24-WAGHOLI BRANCH: Does Not Exist
A44-ANAND BRANCH: Does Not Exist
I41-MANDIDEEP SERVICE CENTER: Does Not Exist
K29-DIAMOND HARBOR SORTING OFFICE: Does Not Exist
T40-HAZIPUR SORTING OFFICE: Does Not Exist
T89-SAGUNA MORE BRANCH: Does Not Exist
U142-BASTI BRANCH: Does Not Exist

Branch names from the list NOT found in df_branch_mapping:
U143-GOMTI NAGAR BRANCH
R11-NAGPUR PANDE LAYOUT BRANCH
R32-NAGPUR GHAT ROAD BRANCH
P24-WAGHOLI BRANCH
I41-MANDIDEEP SERVICE CENTER
T89-SAGUNA MORE BRANCH


## Raod time null analysis

In [ ]:
import pandas as pd


In [ ]:
df1 = pd.read_csv("/content/city_road_time_output.csv")

In [ ]:
df1.head()

,Unnamed: 0,Origin Branch,Destination Branch,Road Time via City
0,0,A01-AHMEDABAD APEX,A01-AHMEDABAD APEX,5.00
1,1,A01-AHMEDABAD APEX,A02-BARODA BRANCH,6.97
2,2,A01-AHMEDABAD APEX,A03-RAJKOT BRANCH,8.86
3,3,A01-AHMEDABAD APEX,A16-KUTCH DDLP SUPER FRANCHISE,12.72
4,4,A01-AHMEDABAD APEX,A47-BHUJ SUPER FRANCHISEE,11.45


In [ ]:
# prompt: fidn all the unique origin branch and destinaition branch for which Road time via City = 0.0, in df1

# Assuming df1 is already loaded and available from the preceding code
# Filter rows where 'Road Time via City' is 0.0
filtered_df = df1[df1['Road Time via City'] == 0.0]

# Find unique combinations of 'Origin Branch' and 'Destination Branch'
unique_od_pairs = filtered_df[['Origin Branch', 'Destination Branch']].drop_duplicates()

print("Unique Origin Branch and Destination Branch pairs where Road Time via City is 0.0:")
print(unique_od_pairs)

# Count the number of such unique pairs
print(f"\nNumber of unique pairs: {len(unique_od_pairs)}")


Unique Origin Branch and Destination Branch pairs where Road Time via City is 0.0:
                        Origin Branch              Destination Branch
155                 A02-BARODA BRANCH               J26-KUNDLI BRANCH
249                 A02-BARODA BRANCH         U143-GOMTI NAGAR BRANCH
274                 A02-BARODA BRANCH  R11-NAGPUR PANDE LAYOUT BRANCH
275                 A02-BARODA BRANCH     R32-NAGPUR GHAT ROAD BRANCH
295                 A02-BARODA BRANCH              P24-WAGHOLI BRANCH
...                               ...                             ...
177982              X35-HGB ROAD APEX          T89-SAGUNA MORE BRANCH
177999              X35-HGB ROAD APEX     W23-BOLANGIR SORTING OFFICE
178003               X37-MORAN BRANCH     X14-DULIAJAN SORTING OFFICE
178026       X38-LANKA SORTING OFFICE              X29-NALBARI BRANCH
178034  X39-SILAPATHAR SORTING OFFICE       X12-IMPHAL SORTING OFFICE

[10947 rows x 2 columns]

Number of unique pairs: 10947


In [ ]:
# prompt: now in this unique_od_pairs, find unique origin branch and Destination branch seoerately

# Find unique Origin Branches from the filtered pairs
unique_origin_branches = unique_od_pairs['Origin Branch'].unique()

# Find unique Destination Branches from the filtered pairs
unique_destination_branches = unique_od_pairs['Destination Branch'].unique()

print("\nUnique Origin Branches where Road Time via City is 0.0:")
print(unique_origin_branches)

print("\nUnique Destination Branches where Road Time via City is 0.0:")
print(unique_destination_branches)

print(f"\nNumber of unique Origin Branches: {len(unique_origin_branches)}")
print(f"Number of unique Destination Branches: {len(unique_destination_branches)}")


Unique Origin Branches where Road Time via City is 0.0:
['A02-BARODA BRANCH' 'A03-RAJKOT BRANCH' 'A04-SURAT APEX'
 'A05-VAPI BRANCH' 'A07-ANKLESHWAR BRANCH' 'A11-KHOKHRA BRANCH'
 'A16-KUTCH DDLP SUPER FRANCHISE' 'A17-BARODA SCS APEX'
 'A18-MEHSANA BRANCH' 'A24-AHMEDABAD BRANCH'
 'A27-BHAVNAGAR SORTING OFFICE' 'A33-JAMNAGAR BRANCH'
 'A36-AHMEDABAD WEST BRANCH' 'A38-GANDHINAGAR BRANCH' 'A44-ANAND BRANCH'
 'A45-SURENDRANAGAR BRANCH' 'A46-SURAT CITY BRANCH'
 'A47-BHUJ SUPER FRANCHISEE' 'A48-CHILODA BRANCH' 'A50-SANAND BRANCH'
 'B01-MANGALORE SUPER FRANCHISEE' 'B02-MYSORE BRANCH' 'B03-HUBLI BRANCH'
 'B05-GULBARGA SUPER FRANCHISEE' 'B06-BOMMANAHALLI BRANCH'
 'B07-CHAMRAJPET BRANCH' 'B08-PEENYA BRANCH' 'B09-RAJAJINAGAR BRANCH'
 'B10-BANGALORE YELAHANKA APEX' 'B11-HENNUR BRANCH'
 'B13-CHITRADURGA SORTING OFFICE' 'B16-BELGAUM SUPER FRANCHISEE'
 'B18-HASSAN BRANCH' 'B19-HOSPET SUPER FRANCHISEE' 'B26-KOGILU BRANCH'
 'B27-HOSUR BRANCH' 'B32-GOA PANJIM BRANCH' 'B35-WHITE FIELD BRANCH'
 'B36-BANGAL

In [ ]:
# prompt: now take the common ones between the Unieque origin branches and destination branches and check which of them does not exist in the df_branch_mapping

common_branches = set(unique_origin_branches) & set(unique_destination_branches)

print("\nBranches that appear as BOTH unique Origin Branches and unique Destination Branches (where road time is 0):")
print(common_branches)

# Check which of these common branches do not exist in df_branch_mapping['Branch_ID_Name']
branches_not_in_mapping = [
    branch for branch in common_branches
    if branch not in df_branch_mapping['Branch_ID_Name'].values
]

print("\nCommon branches (Origin AND Destination where road time is 0) that do NOT exist in df_branch_mapping:")
if branches_not_in_mapping:
    for branch in branches_not_in_mapping:
        print(branch)
    print(f"\nNumber of such branches: {len(branches_not_in_mapping)}")
else:
    print("All common branches (Origin AND Destination where road time is 0) exist in df_branch_mapping.")




Branches that appear as BOTH unique Origin Branches and unique Destination Branches (where road time is 0):
{'J54-KHARAR BRANCH', 'E07-DINDIGUL BRANCH', 'T87-GODDA BRANCH', 'B41-BAGALKOT SORTING OFFICE', 'P10-SANGLI SUPER FRANCHISEE', 'P08-AHMEDNAGAR SUPER FRANCHISEE', 'M48-ANDHERI BRANCH', 'M06-GHATKOPAR BRANCH', 'P22-WARJE BRANCH', 'R05-NAGPUR T/S APEX', 'K40-KHARAGPUR BRANCH', 'T02-RANCHI APEX', 'K65-KOLKATA HP APEX', 'B13-CHITRADURGA SORTING OFFICE', 'O11-THIRUVALLA BRANCH', 'M46-VASAI BRANCH', 'A46-SURAT CITY BRANCH', 'U77-SHAHJAHANPUR SUPER FRANCHISEE', 'R02-CHANDRAPUR SUPER FRANCHISE', 'O23-COCHIN BRANCH', 'C02-PONDICHERRY BRANCH', 'P16-LATUR BRANCH', 'B61-NERALURU BRANCH', 'P65-RANJANGAON BRANCH', 'B50-DAVANGERE BRANCH', 'X21-DIBRUGARH APEX', 'U23-RUDRAPUR BRANCH', 'X30-DHARMANAGAR BRANCH', 'J03-LUDHIANA BRANCH', 'M10-MUMBAI SAKINAKA APEX', 'Q17-MANSAROVAR BRANCH', 'X01-DHARAPUR APEX', 'W02-CUTTACK BRANCH', 'B64-CHIKMAGALUR BRANCH', 'H17-WARANGAL SUPER FRANCHISEE', 'K22-ASANSO

In [ ]:
# prompt: for all these branhces update the mapping that is branch city based on below rules:
# I41-MANDIDEEP SERVICE CENTER    -  Indore
# R32-NAGPUR GHAT ROAD BRANCH   - Nagpur
# P24-WAGHOLI BRANCH - Pune
# R01-NAGPUR AIR APEX - Nagpur
# T89-SAGUNA MORE BRANCH - Patna
# R11-NAGPUR PANDE LAYOUT BRANCH - Nagpur
# U143-GOMTI NAGAR BRANCH - Lucknow
# keep them in capital letters
# P24-WAGHOLI BRANCH
# R01-NAGPUR AIR APEX
# T89-SAGUNA MORE BRANCH
# R11-NAGPUR PANDE LAYOUT BRANCH
# U143-GOMTI NAGAR BRANCH

branch_city_mapping_updates = {
    "I41-MANDIDEEP SERVICE CENTER": "INDORE",
    "R32-NAGPUR GHAT ROAD BRANCH": "NAGPUR",
    "P24-WAGHOLI BRANCH": "PUNE",
    "R01-NAGPUR AIR APEX": "NAGPUR", # Assuming this is a branch-like entity or should be mapped. R01 is also a hub, clarifying context needed. Mapping as requested.
    "T89-SAGUNA MORE BRANCH": "PATNA",
    "R11-NAGPUR PANDE LAYOUT BRANCH": "NAGPUR",
    "U143-GOMTI NAGAR BRANCH": "LUCKNOW"
}

# Apply the updates to df_branch_mapping
for branch_id_name, city in branch_city_mapping_updates.items():
    df_branch_mapping.loc[df_branch_mapping['Branch_ID_Name'] == branch_id_name, 'Branch City'] = city

print("Updated df_branch_mapping head:")
print(df_branch_mapping.head())
# Verify the updates for the specified branches
print("\nVerification of updated branches:")
updated_branches_df = df_branch_mapping[df_branch_mapping['Branch_ID_Name'].isin(branch_city_mapping_updates.keys())]
updated_branches_df

Updated df_branch_mapping head:
       Branch_ID_Name Branch City
0  A01-AHMEDABAD APEX   AHMEDABAD
1   A02-BARODA BRANCH      BARODA
2   A03-RAJKOT BRANCH      RAJKOT
3      A04-SURAT APEX       SURAT
4     A05-VAPI BRANCH        VAPI

Verification of updated branches:


,Branch_ID_Name,Branch City


In [ ]:
df_branch_mapping.head(20)

,Branch_ID_Name,Branch City
0,A01-AHMEDABAD APEX,AHMEDABAD
1,A02-BARODA BRANCH,BARODA
2,A03-RAJKOT BRANCH,RAJKOT
3,A04-SURAT APEX,SURAT
4,A05-VAPI BRANCH,VAPI
5,A07-ANKLESHWAR BRANCH,ANKLESHWAR
6,A11-KHOKHRA BRANCH,AHMEDABAD
7,A16-KUTCH DDLP SUPER FRANCHISE,KUTCH
8,A17-BARODA SCS APEX,BARODA
9,A18-MEHSANA BRANCH,MEHSANA


In [ ]:
# prompt: rename the column names Branch_ID_Name to Branch Name, and Branch City to City

df_branch_mapping = df_branch_mapping.rename(columns={'Branch_ID_Name': 'Branch Name', 'Branch City': 'City'})
print(df_branch_mapping.head())

          Branch Name       City
0  A01-AHMEDABAD APEX  AHMEDABAD
1   A02-BARODA BRANCH     BARODA
2   A03-RAJKOT BRANCH     RAJKOT
3      A04-SURAT APEX      SURAT
4     A05-VAPI BRANCH       VAPI


In [ ]:
df_branch_mapping.to_csv('Branch-city-mapping.csv')

In [ ]:
import pandas as pd

# Modified function to accept the branch_city_mapping DataFrame directly
def compute_city_to_city_road_time(
    shipment_file,
    city_distance_file,
    branch_city_map_df, # Accept DataFrame here
    output_file="city_to_city_road_time.csv"
):
    print("📥 Loading files...")
    df = pd.read_csv(shipment_file)
    distance_df = pd.read_csv(city_distance_file)
    # Removed: branch_city_map = pd.read_csv(branch_city_mapping_file)

    print("⚙️ Building lookup dictionaries...")
    # Build branch → city mapping from the passed DataFrame
    branch_city_lookup = {
        row['Branch Name'].strip(): row['City'].strip()
        for _, row in branch_city_map_df.iterrows() # Use the DataFrame
    }

    # Normalize distance matrix
    distance_lookup = {
        (row['origin'].strip(), row['destination'].strip()): row['duration_hr']
        for _, row in distance_df.dropna(subset=['origin', 'destination', 'duration_hr']).iterrows()
    }

    print("🚦Computing times...")
    road_times = []
    reasons = []

    for _, row in df.iterrows():
        ob, db = row['Origin Branch'].strip(), row['Destination Branch'].strip()

        origin_city = branch_city_lookup.get(ob)
        dest_city = branch_city_lookup.get(db)

        if origin_city is None or dest_city is None:
            road_times.append(None)
            reasons.append("Missing branch-city mapping")
            continue

        base_time = distance_lookup.get((origin_city, dest_city)) or distance_lookup.get((dest_city, origin_city))

        if base_time is None:
            road_times.append(None)
            reasons.append("Missing city-to-city time")
            continue

        total_time = base_time  # Adding 5 hours for hub processing
        road_times.append(total_time)
        reasons.append("OK")

    df['Road Time via City'] = road_times
    df['Road Time Status'] = reasons

    df.to_csv(output_file, index=False)
    print(f"✅ Road time saved to {output_file}")
    print(df[['Origin Branch', 'Destination Branch', 'Road Time via City', 'Road Time Status']].head(5))


# === Example Runner ===
if __name__ == "__main__":
    # Pass the DataFrame variable df_branch_mapping directly
    compute_city_to_city_road_time(
        shipment_file="/content/Daily_datafor_(EP+BP)_ES_path_required_(with_new_rows).csv",
        city_distance_file="/content/city_distances_google.csv",
        branch_city_map_df= pd.read_csv('/content/Branch-city-mapping.csv'), # Pass the DataFrame
        output_file="city_road_time_output.csv"
    )

📥 Loading files...
⚙️ Building lookup dictionaries...
🚦Computing times...
✅ Road time saved to city_road_time_output.csv
        Origin Branch     Destination Branch  Road Time via City  \
0  A01-AHMEDABAD APEX        A05-VAPI BRANCH                6.19   
1  A01-AHMEDABAD APEX  A07-ANKLESHWAR BRANCH                3.17   
2  A01-AHMEDABAD APEX     A11-KHOKHRA BRANCH                0.00   
3  A01-AHMEDABAD APEX    A17-BARODA SCS APEX                1.97   
4  A01-AHMEDABAD APEX     A18-MEHSANA BRANCH                1.72   

  Road Time Status  
0               OK  
1               OK  
2               OK  
3               OK  
4               OK  


In [ ]:
# prompt: keep only Origin Branch, Destination Branch and Road Time via City and Road Time Status columns and make a new dataframe

import pandas as pd
df_city_road = pd.read_csv('/content/city_road_time_output.csv')

df_city_road_subset = df_city_road[['Origin Branch', 'Destination Branch', 'Road Time via City', 'Road Time Status']].copy()

print(df_city_road_subset.head())

df_city_road_subset = df_city_road_subset.drop_duplicates().reset_index(drop=True)
print("\nDataFrame after dropping duplicates:")
print(df_city_road_subset.head())
print("\nDataFrame shape after dropping duplicates:", df_city_road_subset.shape)


        Origin Branch     Destination Branch  Road Time via City  \
0  A01-AHMEDABAD APEX        A05-VAPI BRANCH                6.19   
1  A01-AHMEDABAD APEX  A07-ANKLESHWAR BRANCH                3.17   
2  A01-AHMEDABAD APEX     A11-KHOKHRA BRANCH                0.00   
3  A01-AHMEDABAD APEX    A17-BARODA SCS APEX                1.97   
4  A01-AHMEDABAD APEX     A18-MEHSANA BRANCH                1.72   

  Road Time Status  
0               OK  
1               OK  
2               OK  
3               OK  
4               OK  

DataFrame after dropping duplicates:
        Origin Branch     Destination Branch  Road Time via City  \
0  A01-AHMEDABAD APEX        A05-VAPI BRANCH                6.19   
1  A01-AHMEDABAD APEX  A07-ANKLESHWAR BRANCH                3.17   
2  A01-AHMEDABAD APEX     A11-KHOKHRA BRANCH                0.00   
3  A01-AHMEDABAD APEX    A17-BARODA SCS APEX                1.97   
4  A01-AHMEDABAD APEX     A18-MEHSANA BRANCH                1.72   

  Road Time Status

In [ ]:
df_city_road_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161258 entries, 0 to 161257
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Origin Branch       161258 non-null  object 
 1   Destination Branch  161258 non-null  object 
 2   Road Time via City  129282 non-null  float64
 3   Road Time Status    161258 non-null  object 
dtypes: float64(1), object(3)
memory usage: 4.9+ MB


In [ ]:
# prompt: find the unqiue Origin Branch  and Destination Branch for which Road Time via City is 0.0

# Assuming df1 is already loaded from the preceding code
# Filter rows where 'Road Time via City' is 0.0
filtered_df = df_city_road_subset[df_city_road_subset['Road Time via City'] == 0.0]

# Find unique combinations of 'Origin Branch' and 'Destination Branch'
unique_od_pairs = filtered_df[['Origin Branch', 'Destination Branch']].drop_duplicates()

print("Unique Origin Branch and Destination Branch pairs where Road Time via City is 0.0:")
print(unique_od_pairs)

# Count the number of such unique pairs
print(f"\nNumber of unique pairs: {len(unique_od_pairs)}")

# Find unique Origin Branches from the filtered pairs
unique_origin_branches = unique_od_pairs['Origin Branch'].unique()

# Find unique Destination Branches from the filtered pairs
unique_destination_branches = unique_od_pairs['Destination Branch'].unique()

print("\nUnique Origin Branches where Road Time via City is 0.0:")
print(unique_origin_branches)

print("\nUnique Destination Branches where Road Time via City is 0.0:")
print(unique_destination_branches)

print(f"\nNumber of unique Origin Branches: {len(unique_origin_branches)}")
print(f"Number of unique Destination Branches: {len(unique_destination_branches)}")


Unique Origin Branch and Destination Branch pairs where Road Time via City is 0.0:
                   Origin Branch         Destination Branch
2             A01-AHMEDABAD APEX         A11-KHOKHRA BRANCH
5             A01-AHMEDABAD APEX       A24-AHMEDABAD BRANCH
8             A01-AHMEDABAD APEX  A36-AHMEDABAD WEST BRANCH
13            A01-AHMEDABAD APEX         A48-CHILODA BRANCH
1858         A17-BARODA SCS APEX          A02-BARODA BRANCH
...                          ...                        ...
159248       X36-AMINGAON BRANCH     X10-LANUNG GAON BRANCH
159623       X36-AMINGAON BRANCH          X01-DHARAPUR APEX
159636       X36-AMINGAON BRANCH          X22-DISPUR BRANCH
160676  X38-LANKA SORTING OFFICE          X19-NAGAON BRANCH
160684  X38-LANKA SORTING OFFICE        X33-JAGIROAD BRANCH

[597 rows x 2 columns]

Number of unique pairs: 597

Unique Origin Branches where Road Time via City is 0.0:
['A01-AHMEDABAD APEX' 'A17-BARODA SCS APEX' 'B06-BOMMANAHALLI BRANCH'
 'B07-CHAMRAJPET 

In [ ]:
df_city_road_subset.to_csv('final_road_time_calc.csv')

## Merging all cases

In [ ]:
import pandas as pd

# === Load case files ===
case1 = pd.read_csv("/content/Case1_indirect_for_left_of_the_rows.csv")
case2 = pd.read_csv("/content/case2_indirect_for_left_of_the_rows.csv")
case3 = pd.read_csv("/content/case3_routes_output_new2.csv")
case4 = pd.read_csv("/content/Case4_direct_air_routes_output.csv")
road_time = pd.read_csv("/content/final_road_time_calc.csv")

# === Standardize branch names ===
def standardize(df):
    df["Origin Branch"] = df["Origin Branch"].astype(str).str.strip().str.upper()
    df["Destination Branch"] = df["Destination Branch"].astype(str).str.strip().str.upper()
    return df

case1 = standardize(case1)
case2 = standardize(case2)
case3 = standardize(case3)
case4 = standardize(case4)
road_time = standardize(road_time)

# === Rename time columns for consistency ===
case1 = case1.rename(columns={"Indirect Total Time": "Indirect Total Time"})
case2 = case2.rename(columns={"Total Time": "Indirect Total Time"})
case3 = case3.rename(columns={"Total Time": "Indirect Total Time"})
case4 = case4.rename(columns={"Total Time": "Indirect Total Time"})

# === Add Source identifiers ===
case1["Source"] = "Case1"
case2["Source"] = "Case2"
case3["Source"] = "Case3"
case4["Source"] = "Case4"

# === Combine all cases ===
combined = pd.concat([case1, case2, case3, case4], ignore_index=True)

# === Create unique route key: Branch + Product ===
combined["Key"] = (
    combined["Origin Branch"] + "|" +
    combined["Destination Branch"] + "|" +
    combined["Product Type"]
)

# === Sort by least Indirect Time and keep best per key ===
combined = combined.sort_values(by="Indirect Total Time", ascending=True)
best_routes = combined.groupby("Key", as_index=False).first()

# === Clean Suggested Route to remove duplicate segments ===
def clean_route(route):
    if pd.isna(route):
        return route
    parts = [p.strip() for p in route.split("→")]
    seen = set()
    cleaned = []
    for p in parts:
        if p not in seen:
            seen.add(p)
            cleaned.append(p)
    return " → ".join(cleaned)

best_routes["Suggested Route"] = best_routes["Suggested Route"].apply(clean_route)

# === Add Road Time ===
road_time["Key"] = (
    road_time["Origin Branch"] + "|" +
    road_time["Destination Branch"]
)
road_map = road_time.set_index("Key")["Road Time via City"].to_dict()

# Recreate matchable key (excluding product type)
best_routes["Road_Key"] = (
    best_routes["Origin Branch"] + "|" +
    best_routes["Destination Branch"]
)
best_routes["Total Road Time"] = best_routes["Road_Key"].map(road_map)

# === Calculate Time Saved and Recommendation ===
best_routes["Time Saved"] = best_routes["Indirect Total Time"] - best_routes["Total Road Time"]
best_routes["Recommendation"] = best_routes["Time Saved"].apply(
    lambda x: "Road" if pd.notna(x) and x > 0 else "Air"
)

# === Final Output Columns ===
final_cols = [
    "Origin Branch", "Destination Branch", "Product Type", "Weight (kg)",
    "Suggested Route", "Indirect Total Time", "Total Volume", "Time_Components",
    "Total Road Time", "Time Saved", "Recommendation"
]
final_output = best_routes[final_cols]

# === Save to CSV ===
final_output.to_csv("final_routes_for_left_over_rows.csv", index=False)
print("✅ Final best routes saved to 'final_routes_for_left_over_rows.csv'")

✅ Final best routes saved to 'final_routes_for_left_over_rows.csv'


In [ ]:
df_new = pd.read_csv('/content/final_routes_for_left_over_rows.csv')

In [ ]:
df_new.head(10)

,Origin Branch,Destination Branch,Product Type,Weight (kg),Suggested Route,Indirect Total Time,Total Volume,Time_Components,Total Road Time,Time Saved,Recommendation
0,A01-AHMEDABAD APEX,A05-VAPI BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ MUMBAI → M10...,16.434128,567.31072,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.50 t6:2.0 t...,6.19,10.244128,Road
1,A01-AHMEDABAD APEX,A07-ANKLESHWAR BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ MUMBAI → M10...,15.594128,567.31072,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.50 t6:2.0 t...,3.17,12.424128,Road
2,A01-AHMEDABAD APEX,A11-KHOKHRA BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,16.719513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,0.00,16.719513,Road
3,A01-AHMEDABAD APEX,A17-BARODA SCS APEX,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,18.499513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,1.97,16.529513,Road
4,A01-AHMEDABAD APEX,A18-MEHSANA BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,17.729513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,1.72,16.009513,Road
5,A01-AHMEDABAD APEX,A24-AHMEDABAD BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,16.749513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,0.00,16.749513,Road
6,A01-AHMEDABAD APEX,A27-BHAVNAGAR SORTING OFFICE,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ MUMBAI → M10...,16.724128,567.31072,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.50 t6:2.0 t...,3.20,13.524128,Road
7,A01-AHMEDABAD APEX,A33-JAMNAGAR BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,21.999513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,5.47,16.529513,Road
8,A01-AHMEDABAD APEX,A36-AHMEDABAD WEST BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,16.829513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,0.00,16.829513,Road
9,A01-AHMEDABAD APEX,A38-GANDHINAGAR BRANCH,EP_BP,0.0,A01-AHMEDABAD APEX → AHMEDABAD ✈️ INDORE → I01...,16.809513,221.25972,t1:0.00 t2:2.5 t3:0.20 t4:2.0 t5:1.42 t6:2.0 t...,0.71,16.099513,Road


## Branch-hub_time calculation

In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from itertools import combinations

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees) using Haversine formula
    Returns distance in kilometers
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))

    # Radius of earth in kilometers
    r = 6371

    return c * r

def estimate_duration(distance_km, avg_speed_kmh=50):
    """
    Estimate travel duration based on distance
    Default average speed: 50 km/h (can be adjusted based on road conditions)
    For Indian roads, you might want to use:
    - Highway: 60-80 km/h
    - City roads: 30-40 km/h
    - Mixed: 50 km/h (default)
    Returns duration in hours
    """
    return distance_km / avg_speed_kmh

# Method 1: Read from your uploaded file
def read_from_file(filename):
    """Read data from uploaded CSV file"""
    try:
        df = pd.read_csv('/content/branch_coordinates_new.csv')
        # Remove index column if it exists
        if df.columns[0].isdigit() or 'Unnamed' in df.columns[0]:
            df = df.drop(df.columns[0], axis=1)
        return df
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

# Method 2: Use the data directly (for demonstration)
def create_sample_data():
    """Create sample data for testing"""
    data = {
        'Branch/Hub Name': [
            'A01-AHMEDABAD APEX', 'A02-BARODA BRANCH', 'A03-RAJKOT BRANCH',
            'A48-CHILODA BRANCH', 'A05-VAPI BRANCH', 'A07-ANKLESHWAR BRANCH',
            'B01-MANGALORE SUPER FRANCHISEE', 'B02-MYSORE BRANCH', 'B03-HUBLI BRANCH',
            'C02-PONDICHERRY BRANCH', 'C08-KATPADI BRANCH', 'C13-GUINDY BRANCH'
        ],
        'Latitude': [
            23.106556, 22.33424307, 22.27983403, 23.106326, 20.36025, 21.611786,
            12.88889313, 12.2854657, 15.356049, 11.946191, 12.93120167, 13.01608
        ],
        'Longitude': [
            72.683222, 73.16552617, 70.79548111, 72.683018, 72.9179, 73.0201776,
            74.82789612, 76.6129104, 75.130127, 79.777866, 79.13423333, 80.21167
        ]
    }
    return pd.DataFrame(data)

def compute_distance_matrix(df, save_to_csv=True, avg_speed=50):
    """
    Compute distance-time matrix for all branch pairs

    Parameters:
    df: DataFrame with columns 'Branch/Hub Name', 'Latitude', 'Longitude'
    save_to_csv: Whether to save results to CSV
    avg_speed: Average speed in km/h for duration calculation

    Returns:
    DataFrame with columns: Branch, Hub, Distance_km, Duration_hr
    """

    # Clean and prepare data
    df = df.copy()
    df.columns = df.columns.str.strip()

    print(f"Processing {len(df)} branches/hubs")

    # Extract data
    branches = df['Branch/Hub Name'].tolist()
    latitudes = df['Latitude'].tolist()
    longitudes = df['Longitude'].tolist()

    # Create result list
    results = []
    total_pairs = len(branches) * len(branches)

    print(f"Computing {total_pairs} distance-time pairs...")

    # Calculate distances for all pairs
    for i, branch1 in enumerate(branches):
        if i % 50 == 0:  # Progress indicator
            print(f"Processing branch {i+1}/{len(branches)}")

        for j, branch2 in enumerate(branches):
            lat1, lon1 = latitudes[i], longitudes[i]
            lat2, lon2 = latitudes[j], longitudes[j]

            # Calculate distance using Haversine formula
            distance = haversine_distance(lat1, lon1, lat2, lon2)

            # Estimate duration
            duration = estimate_duration(distance, avg_speed)

            results.append({
                'Branch': branch1,
                'Hub': branch2,
                'Distance_km': round(distance, 2),
                'Duration_hr': round(duration, 2)
            })

    # Create result DataFrame
    result_df = pd.DataFrame(results)

    # Save to CSV if requested
    if save_to_csv:
        filename = 'distance_time_matrix_full.csv'
        result_df.to_csv(filename, index=False)
        print(f"\nResults saved to '{filename}'")

    return result_df

def analyze_results(result_df):
    """Analyze and display statistics about the distance matrix"""

    print(f"\n=== DISTANCE MATRIX ANALYSIS ===")
    print(f"Total pairs: {len(result_df):,}")
    print(f"Unique branches: {result_df['Branch'].nunique()}")

    # Distance statistics
    print(f"\n--- Distance Statistics ---")
    print(f"Min distance: {result_df['Distance_km'].min()} km")
    print(f"Max distance: {result_df['Distance_km'].max():.2f} km")
    print(f"Mean distance: {result_df['Distance_km'].mean():.2f} km")
    print(f"Median distance: {result_df['Distance_km'].median():.2f} km")

    # Duration statistics
    print(f"\n--- Duration Statistics ---")
    print(f"Min duration: {result_df['Duration_hr'].min()} hours")
    print(f"Max duration: {result_df['Duration_hr'].max():.2f} hours")
    print(f"Mean duration: {result_df['Duration_hr'].mean():.2f} hours")
    print(f"Median duration: {result_df['Duration_hr'].median():.2f} hours")

    # Show longest distances
    print(f"\n--- Top 5 Longest Distances ---")
    longest = result_df.nlargest(5, 'Distance_km')[['Branch', 'Hub', 'Distance_km', 'Duration_hr']]
    print(longest.to_string(index=False))

    # Show same location pairs (should be 0 distance)
    print(f"\n--- Sample Same Location Pairs (Distance = 0) ---")
    same_location = result_df[result_df['Distance_km'] == 0].head(5)
    print(same_location[['Branch', 'Hub', 'Distance_km']].to_string(index=False))

# Main execution
if __name__ == "__main__":

    # Try to read from uploaded file first
    print("Attempting to read from uploaded file...")
    df = read_from_file('paste.txt')  # Replace with your actual filename

    if df is None:
        print("Using sample data for demonstration...")
        df = create_sample_data()

    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())

    # Compute distance matrix
    print(f"\nStarting distance matrix computation...")
    result_df = compute_distance_matrix(df, save_to_csv=True, avg_speed=50)

    # Analyze results
    analyze_results(result_df)

    # Display sample results
    print(f"\n--- Sample Results ---")
    print(result_df.head(10).to_string(index=False))

    print(f"\n=== COMPUTATION COMPLETE ===")
    print(f"Final output format:")
    print("Branch | Hub | Distance_km | Duration_hr")
    print("Results saved to 'distance_time_matrix_full.csv'")

# Additional utility functions
def filter_distance_matrix(result_df, max_distance=None, max_duration=None):
    """Filter results by distance or duration thresholds"""
    filtered = result_df.copy()

    if max_distance:
        filtered = filtered[filtered['Distance_km'] <= max_distance]
        print(f"Filtered to {len(filtered)} pairs within {max_distance} km")

    if max_duration:
        filtered = filtered[filtered['Duration_hr'] <= max_duration]
        print(f"Filtered to {len(filtered)} pairs within {max_duration} hours")

    return filtered

def get_nearby_branches(result_df, branch_name, max_distance=100):
    """Find branches within specified distance of a given branch"""
    nearby = result_df[
        (result_df['Branch'] == branch_name) &
        (result_df['Distance_km'] <= max_distance) &
        (result_df['Distance_km'] > 0)  # Exclude self
    ].sort_values('Distance_km')

    return nearby[['Hub', 'Distance_km', 'Duration_hr']]

Attempting to read from uploaded file...

Dataset shape: (587, 3)
Columns: ['Branch/Hub Name', 'Latitude', 'Longitude']

First few rows:
      Branch/Hub Name   Latitude  Longitude
0  A01-AHMEDABAD APEX  23.106556  72.683222
1   A02-BARODA BRANCH  22.334243  73.165526
2   A03-RAJKOT BRANCH  22.279834  70.795481
3  A48-CHILODA BRANCH  23.106326  72.683018
4     A05-VAPI BRANCH  20.360250  72.917900

Starting distance matrix computation...
Processing 587 branches/hubs
Computing 344569 distance-time pairs...
Processing branch 1/587
Processing branch 51/587
Processing branch 101/587
Processing branch 151/587
Processing branch 201/587
Processing branch 251/587
Processing branch 301/587
Processing branch 351/587
Processing branch 401/587
Processing branch 451/587
Processing branch 501/587
Processing branch 551/587

Results saved to 'distance_time_matrix_full.csv'

=== DISTANCE MATRIX ANALYSIS ===
Total pairs: 344,569
Unique branches: 587

--- Distance Statistics ---
Min distance: 0.0 km
Max 